# Install Required libraries (not finished changing names of models and tokenizer)

In [ ]:
!pip install sacremoses transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.0 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=1b62cb6beb9c4ece286f9ea3003ba44894b741e77139e8bc0b93791482d759cf
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


# Import required libraries

In [ ]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, pipeline, AutoModel
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Import Data

In [ ]:
df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/dissertation/data/2020_noFT.json')
#df = df.drop("Predict", axis=1)
#df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dissertation/data/GOUT-CC-2020-CORPUS-REDACTED.csv')
#df2 = df2.drop("Predict", axis=1)

#df = pd.concat([df, df2], ignore_index=True)

In [ ]:
df.shape

(8437, 3)

In [ ]:
# Check for consensus and mark rows for removal
rows_to_drop = []
for index, row in df.iterrows():
    if row['Consensus'] == '-': #Drop unknown and unmarked
        rows_to_drop.append(index)

# Drop the marked rows
df.drop(rows_to_drop, inplace=True)

# Print the updated DataFrame
print(df)

                                        Chief Complaint Consensus
1     "can't walk", reports onset at <<TIME>>. orien...         N
2     "dehydration" Chest hurts, hips hurt, cramps P...         Y
3     "gout flare up" L arm swelling x 1 week. denie...         Y
5     "I started breathing hard"  hx- htn, gout, anx...         N
6     "I think I have a gout flare up" L wrist pain ...         Y
...                                                 ...       ...
8424  sob and right chest pain x 1 weeks  - hx of mu...         N
8425  starts in lower back and goes right  legs x1 m...         N
8427  sciatica pain R lower back radiating to R groi...         N
8432  stepped on a nail at home with right foot, pai...         N
8436  Rash/sores across body, infection ro left thum...         N

[445 rows x 2 columns]


# Small data analysis

In [ ]:
df.groupby("Consensus").size()
print("Predict is U")
print(df[df['Predict'] == 'U'].groupby('Consensus').size())
print("-----------------------------------------------------------------")
print("Predict is N")
print(df[df['Predict'] == 'N'].groupby('Consensus').size())
print("-----------------------------------------------------------------")
print("Predict is Y")
print(df[df['Predict'] == 'Y'].groupby('Consensus').size())
print("-----------------------------------------------------------------")
print("Predict is -")
print(df[df['Predict'] == '-'].groupby('Consensus').size())
print("-----------------------------------------------------------------")


Predict is U
Consensus
-      2
N    127
U     12
Y     15
dtype: int64
-----------------------------------------------------------------
Predict is N
Consensus
-    7955
N     201
U       4
Y       8
dtype: int64
-----------------------------------------------------------------
Predict is Y
Consensus
-    17
N    22
Y    72
dtype: int64
-----------------------------------------------------------------
Predict is -
Consensus
-    2
dtype: int64
-----------------------------------------------------------------


# Load Model and tokenizer (BioGPT)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#GPT-2 is a model with absolute position embeddings so it's usually advised to pad the inputs on the right rather than the left. Remember this
tokenizer = GPT2Tokenizer.from_pretrained("stanford-crfm/BioMedLM")
tokenizer.padding_side = "left"

model = AutoModel.from_pretrained("stanford-crfm/BioMedLM")

#model = model.to(device)

Some weights of the model checkpoint at stanford-crfm/BioMedLM were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at stanford-crfm/BioMedLM and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#clear GPU cache
#torch.cuda.empty_cache()

## Define the pipeline

In [ ]:
# Create the pipeline
p = pipeline(
    task="feature-extraction",
    tokenizer="stanford-crfm/BioMedLM",
    model="stanford-crfm/BioMedLM",
    framework="pt",
    device=0,  # use CUDA with 0
)

Some weights of the model checkpoint at stanford-crfm/BioMedLM were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Get the embeddings of the last token of the last hidden state

In [ ]:
def extract_last_token(last_hidden_states):
    last_hidden_states = np.array(last_hidden_states)
    return last_hidden_states[:,-1,:]

# Process the data using the pipeline
results = p([row["Chief Complaint"] for _, row in df.iterrows()])

# Extract the last token of the last hidden state
embeddings = [extract_last_token(last_hidden_state ) for last_hidden_state  in results]

In [ ]:
# Assign the reshaped embeddings to the "embeddings" column in the DataFrame
df["emb_Stanford_no_FineT"] = embeddings

# Print the resulting DataFrame
print(df)

                                       Chief Complaint Predict Consensus  \
0    "been feeling bad" last 2 weeks & switched BP ...       N         -   
1    "can't walk", reports onset at <<TIME>>. orien...       Y         N   
2    "dehydration" Chest hurts, hips hurt, cramps P...       Y         Y   
3    "gout flare up" L arm swelling x 1 week. denie...       Y         Y   
4    "heart racing,"dyspnea, and orthopnea that has...       N         -   
..                                                 ...     ...       ...   
295  upper abd/R side chest pain x1 month, new onse...       N         N   
296  upper lip swelling x one day, pmh HTN, COPD, b...       N         N   
297  walked outside of a gas station and began bein...       N         -   
298  was getting prepped for colonoscopy and was se...       N         N   
299  Was seen at <<HOSPITAL>> after an MVC. Pt stat...       N         N   

                                   emb_biogpt_no_FineT  \
0    [[0.4998164773, -1.14541

## Save to csv ot txt

In [ ]:
df.to_json('/content/drive/MyDrive/Colab Notebooks/dissertation/data/2020_noFT.json', orient='records')